In [41]:
# Import the necessary dependencies
from IPython.display import display

# Operating System
import os

# Numpy, Pandas and Scipy
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, coo_matrix, save_npz, load_npz

# Scikit-learn
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

# Model Evaluation
#from evaluation import evaluate

In [4]:
print("train_reviews.csv \n")
!head -3 data/train_reviews.csv

train_reviews.csv 

,product_id,review_text,review_date,user_id,summary,overall
0,1881509818,"This came in on time and I am veru happy with it, I haved used it already and it makes taking out the pins in my glock 32 very easy",2014-01-26,AIXZKN4ACSKI,Woks very good,5
1,1881509818,"I had a factory Glock tool that I was using for my Glock 26, 27, and 17.  I've since lost it and had needed another.  Since I've used Ghost products prior, and know that they are reliable, I had decided to order this one.  Sure enough, this is just as good as a factory tool.",2012-02-02,A1L5P841VIO02V,Works as well as the factory tool,5


In [5]:
print("train_products_metadata.csv\n")
!head -4 data/train_products_metadata.csv

train_products_metadata.csv

,product_id,brand,categories,description,price
0,1881509818,Ghost,"['Sports & Outdoors', 'Hunting & Fishing', 'Hunting', 'Gun Maintenance', 'Gunsmithing Tools']","Ghost Armorer Tool (1). The GAT is made with a spring steel punch. The diameter is 3/32 of an inch or 2.5mm, this is the same as the OEM tool size. The difference is you will be able to press harder without bending the shaft of this punch. Just a better tool to work on your Glock with.",9.99
1,2094869245,,"['Sports & Outdoors', 'Cycling', 'Lights & Reflectors', 'Taillights']","This newly-designed Laser tail light can emit two parallel lines, to form a virtual lane together with the moving of bicycle on the road. LED flash light and 
two lines not only enhance the waring effect strongly and greatly but also improve the safety of night riding.",8.26


In [6]:
print("test_users.csv \n")
!head -3 data/test_users.csv

test_users.csv 

user_id
A0029274J35Q1MYNKUWO
A0103849GBVWICKXD4T6


In [73]:
print("sample_submission.csv \n")
! head -3 data/sample_submission.csv

sample_submission.csv 

user_id,purchases
A0029274J35Q1MYNKUWO,"[""B001E0O6BY"", ""B0048W2FK4"", ""B0038R9UOE"", ""B004N62I6G"", ""B0030IASWE"", ""B000FNB006"", ""B002LARNO8"", ""B001JJCHUC"", ""B000FSSQRG"", ""B003CVQZ44"", ""B00097DJH0"", ""B007PIZLZ4"", ""B001ASJDWW"", ""B004SOPJG4"", ""B0055OEZDY"", ""B004TADJ1Y"", ""B000BO1OKA"", ""B0018UXX46"", ""B0014VX5XI"", ""B001BAH0V0"", ""B001QDR2SI"", ""B001P69PWM"", ""B007K63SEC"", ""B004LRBKPC"", ""B000HCRKTA"", ""B0007M8G2K"", ""B0009GA6SQ"", ""B006YBWXK4"", ""B001WJ58GE"", ""B003SHBL00"", ""B003K827PU"", ""B00165Q962"", ""B0007MZV3C"", ""B000XKHHMG"", ""B0037H3EB0"", ""B004D32436"", ""B004WF51Q2"", ""B006WIW6H4"", ""B0023F9YTS"", ""B004GBKYJG"", ""B001M5TZZI"", ""B001CJXB5E"", ""B009V5HXGO"", ""B0006ZK3XI"", ""B000RQETTK"", ""B0012QJW6E"", ""B00869YGV6"", ""B000GU6PZ8"", ""B003JWEUI4"", ""B003X5D9DY""]"
A0103849GBVWICKXD4T6,"[""B000GK9XXY"", ""B0080MPWSA"", ""B00C6KVAR4"", ""B00270D5GC"", ""B000C14K30"", ""B0006IRSK6"", ""B0014VVHWY"",

In [79]:
pd.read_csv("testy.csv").purchases.iloc[0]

"['1', '5']"

# Load the Train and Test files

In [59]:
def read_users_reviews() -> pd.DataFrame:
    """Imports the review history for each user.
    
    Returns:
        data (pd.DataFrame): DataFrame with the user for each user. 
                             The rows are tuples of (user, song_id, rating).
                             
    """
    path = os.path.join('data', 'train_reviews.csv')
    data = pd.read_csv(path).drop(columns='Unnamed: 0').rename(columns={'user_id':'user'})
    data['review_date'] = pd.to_datetime(data['review_date'])
    return data

data = read_users_reviews()
display(data.shape)
data.head()

(268169, 6)

,product_id,review_text,review_date,user,summary,overall
0,1881509818,This came in on time and I am veru happy with ...,2014-01-26,AIXZKN4ACSKI,Woks very good,5
1,1881509818,I had a factory Glock tool that I was using fo...,2012-02-02,A1L5P841VIO02V,Works as well as the factory tool,5
2,1881509818,If you don't have a 3/32 punch or would like t...,2012-02-28,AB2W04NI4OEAD,"It's a punch, that's all.",4
3,1881509818,This works no better than any 3/32 punch you w...,2012-02-05,A148SVSWKTJKU6,It's a punch with a Glock logo.,4
4,1881509818,I purchased this thinking maybe I need a speci...,2013-04-23,AAAWJ6LW9WMOO,"Ok,tool does what a regular punch does.",4


In [60]:
data.isnull().sum()

product_id       0
review_text    115
review_date      0
user             0
summary          0
overall          0
dtype: int64

Let's handle the missing values

In [61]:
data['review_text'] = data['review_text'].fillna('')

In [62]:
print(f"We have {len(data):,} ratings in total.")
print(f'We have a history of {data.user.nunique()} unique users in total.')

We have 268,169 ratings in total.
We have a history of 34713 unique users in total.


In [63]:
def read_test_users() -> pd.DataFrame:
    """Imports the list of users for which we need to predict.
    
    Returns:
        users_to_pred (pd.DataFrame): DataFrame with the users for which we will recommend songs. 
    """

    path = os.path.join('data', 'test_users.csv')
    users_to_pred_ = pd.read_csv(path).rename(columns={'user_id': 'users to recommend songs'})
    
    return users_to_pred_


users_to_pred = read_test_users()
display(users_to_pred.shape)
users_to_pred.head()

(9225, 1)

,users to recommend songs
0,A0029274J35Q1MYNKUWO
1,A0103849GBVWICKXD4T6
2,A01685981QK9IX1Q16YZY
3,A02904661A62AP64S46MT
4,A036147939NFPC389VLK


In [64]:
print(f"We have {len(users_to_pred)} users in need for better songs!")

We have 9225 users in need for better songs!


In [65]:
def get_indices_from_users_to_pred(users_to_pred: pd.DataFrame, data: pd.DataFrame):
    """Get the indices of users_to_pred for which we have data and for which we don't.
    
    Args:
        users_to_pred (pd.DataFrame): DataFrame containing the list of users we are going to recommend items.
        data (pd.DataFrame): Original of listening history for the users.
        
    Returns:
        index_users_in_data (Int64Index): Index that filters the users_to_pred to get the user_id's with training data.
        index_users_not_in_data (Int64Index): Index that filters the users_to_pred to get the user_id's without training data.
        
    """
    index_users_in_data = users_to_pred[users_to_pred.isin(data.user.values).values].index
    index_users_not_in_data = users_to_pred[~users_to_pred.isin(data.user.values).values].index
    
    return index_users_in_data, index_users_not_in_data


index_users_in_data, index_users_not_in_data = get_indices_from_users_to_pred(users_to_pred, data)

In [66]:
# For further inspection, we advise you to look at the objects themselves.
print(f"The index for users which we have training data has length of {len(index_users_in_data)}.")
print(f"The index for users which we don't have training data has length of {len(index_users_not_in_data)}.")

The index for users which we have training data has length of 8343.
The index for users which we don't have training data has length of 882.


In [67]:
def get_users_to_pred_by_index(users_to_pred, index_users_in_data):
    """DataFrame with user_id's in test set for for which we have training data.

    Args: 
        users_to_pred (pd.DataFrame): DataFrame containing the list of users we are going to recommend items.
        index_users_in_data (Int64Index): Index that filters the users_to_pred to get the user_id's with training data.
    Returns:
        users_in_data (pd.DataFrame): Dataframe containing the list of user_id's with training data.
    
    """
    return users_to_pred.iloc[index_users_in_data].reset_index(drop=True)

# Get the test users with training data
test_users_in_data = get_users_to_pred_by_index(users_to_pred, index_users_in_data)
print(test_users_in_data.shape)
test_users_in_data.head()

(8343, 1)


,users to recommend songs
0,A0029274J35Q1MYNKUWO
1,A0103849GBVWICKXD4T6
2,A01685981QK9IX1Q16YZY
3,A02904661A62AP64S46MT
4,A036147939NFPC389VLK


In [68]:
def make_ratings(data: pd.DataFrame) -> csr_matrix:
    """Creates the ratings matrix of listening history.
    
    Creates the ratings matrix from the listening history imported using the read_users_history() method.
    
    Args:
        data (pd.DataFrame):  listening history for the users.
        
    Returns:
        ratings (csr_matrix): ratings matrix with shape (n_users, n_items)
        
    TODO:
        * Add the shape as an optional parameter.
        
    """
    # You have probably seen this before
    users, user_pos = np.unique(data['user'].values, return_inverse=True)
    items, item_pos = np.unique(data['product_id'].values, return_inverse=True)
    values = data['overall'].fillna(0).values
    
    shape = (len(users), len(items))

    R_ = coo_matrix((values, (user_pos, item_pos)), shape=shape,dtype='int16').tocsr()
    
    return R_


R = make_ratings(data)
R

<34713x18264 sparse matrix of type '<class 'numpy.int16'>'
	with 268169 stored elements in Compressed Sparse Row format>

In [69]:
# Just for mental (in)sanity, let's match the info of the matrix to what is printed in the previous cell.
print(f"The shape is {R.shape}")
print(f"The dtypes of the elements are {R.dtype}")
print(f"The number of stored elements is {R.nnz}")
print(f"The type of the matrix is {type(R)}")

The shape is (34713, 18264)
The dtypes of the elements are int16
The number of stored elements is 268169
The type of the matrix is <class 'scipy.sparse.csr.csr_matrix'>


Note, changed dtype to int16 to ocupy less space

In [70]:
# Let's store a Series with the unique user id's that we have in the original data.
def get_unique_users(data):
    """Get unique users in training data.
    
    Args:
        data (pd.DataFrame):  listening history for the users.
        
    Returns:
        unique_users (pd.DataFrame): DataFrame of one column with unique users in training data.
    
    """
    return pd.DataFrame(np.unique(data['user'].values), columns=["users to recommend songs"])


unique_users_training_data = get_unique_users(data)
unique_users_training_data.head()

,users to recommend songs
0,A00046902LP5YSDV0VVNF
1,A0029274J35Q1MYNKUWO
2,A004507634IQ8TNQ3YTUI
3,A005011233SVRED9Q0VY0
4,A00676013IBZZQ3UYWK47


In [71]:
def get_indices_in_ratings_for_test_users_in_data(data: pd.DataFrame, test_users_in_data: pd.DataFrame):
    """Returns the index of the ratings matrix for the test users for which we have training data.
    
    Args:
        data (pd.DataFrame): DataFrame with the user for each user. 
                             The rows are tuples of (user, song_id, rating).
                             
        test_users_in_data (pd.DataFrame): DataFrame containing the list of test users for which we have training data.
        
    Returns:
        indices_ratings_tests_users_in_data (np.array): Indices of users in test set with training data for ratings matrix.
    
    """
    unique_users = get_unique_users(data).iloc[:, 0]
    indices_ratings_tests_users_in_data = unique_users[unique_users.isin(test_users_in_data.iloc[:, 0])].index.to_numpy()                                                                                       
    return indices_ratings_tests_users_in_data

indices_ratings_tests_users_in_data = get_indices_in_ratings_for_test_users_in_data(data, test_users_in_data)

In [58]:
test_size = 0.2
idx
data

,product_id,review_text,review_date,user,summary,overall
0,1881509818,This came in on time and I am veru happy with ...,2014-01-26,AIXZKN4ACSKI,Woks very good,5
1,1881509818,I had a factory Glock tool that I was using fo...,2012-02-02,A1L5P841VIO02V,Works as well as the factory tool,5
2,1881509818,If you don't have a 3/32 punch or would like t...,2012-02-28,AB2W04NI4OEAD,"It's a punch, that's all.",4
3,1881509818,This works no better than any 3/32 punch you w...,2012-02-05,A148SVSWKTJKU6,It's a punch with a Glock logo.,4
4,1881509818,I purchased this thinking maybe I need a speci...,2013-04-23,AAAWJ6LW9WMOO,"Ok,tool does what a regular punch does.",4
5,1881509818,"Needed this tool to really break down my G22, ...",2012-11-02,A2XX2A4OJCDNLZ,Glock punch tool - needed for your Glock and o...,5
6,2094869245,This light will no doubt capture the attention...,2013-08-31,AWG3H90WVZ0Z1,Bright!,4
7,2094869245,"Light and laser torch work well, very bright. ...",2013-05-27,A3V52OTJHKIJZX,Be seen,5
8,2094869245,Does everything it says it will do. I would li...,2013-11-02,A3SZBE5F3UQ9EC,Bicycle rear tail light,5
9,2094869245,It's cheaply made but does what it is supposed...,2014-01-07,A21AJ9GNCM89MK,It's worth the price they charge.,3


In [ ]:
# Percentage of listening history used for validation.
test_size = 0.2

def make_train_val_split(data: pd.DataFrame, test_size : float = 0.2, shape: tuple = None):
    """Split the data into train and validation and returns the ratings matrixes accordingly.
    
    Args:
        data (pd.DataFrame): Listening history for the users.
        test_size (float): Percentage of listening history used for validation.
        shape (tuple): The overall (n_users, n_items) shape desired for the matrix. 
                       If None, define the shape with the (n_users, n_items) from 'data' argument.
    
    Returns:
        ratings_train (csr_matrix): Ratings matrix for train.
        ratings_val (csr_matrix): Ratings matrix for validation.
    
    """
    
    def b 
    train_data, val_data = train_test_split(data, test_size=test_size, random_state=8)
    R_train = make_ratings(train_data, shape=shape)
    R_val = make_ratings(val_data, shape=shape)
    return R_train, R_val

ratings_train, ratings_val = make_train_val_split(data, test_size=test_size, shape=R.shape)